# DI725 Assignment 1 - Preprocessing - Modeling - Evaluation

Ali Yiğit Başaran - 2231355

In [1]:
# Installing dependencies for training and augmentation, preprocessing
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install torch numpy transformers datasets tiktoken wandb tqdm
!pip install torchmetrics
!pip install imbalanced-learn
!pip install nlpaug numpy nltk
!pip install nbformat
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
!pip install -U wandb

Looking in indexes: https://download.pytorch.org/whl/cu121



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ali\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ali\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!git clone https://github.com/caglarmert/DI725.git

fatal: destination path 'DI725' already exists and is not an empty directory.


In [3]:
%cd DI725/assignment_1

c:\Users\Ali\Desktop\DI725_Asg1_Repo2\DI725_Assignment1\DI725\assignment_1


C:\Users\Ali\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
# General
import os, time, math, pickle, io, inspect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from contextlib import nullcontext
from dataclasses import dataclass
import wandb
from contextlib import nullcontext

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
from torchmetrics import MeanMetric

# HuggingFace and SMOTE
from transformers import GPT2Tokenizer
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Custom model and dataset
from model import GPT, GPTConfig

import nltk
nltk.download('averaged_perceptron_tagger_eng')


c:\Users\Ali\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Ali\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

# Preprocessing and Modeling

In [5]:
# Creating GPT sentiment class to convert decoder only GPT2 model to sentiment analysis model
import torch.nn as nn
import torch
from model import GPT
class GPTSentiment(GPT):
    def __init__(self, config):
        super().__init__(config)
        # We will add 3 classes for positive,neural and negative
        self.sentiment_head = nn.Linear(config.n_embd, 3)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        # Assertion for block size (exists in original repo)
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"

        pos = torch.arange(0, t, dtype=torch.long, device=device)
        tok_emb = self.transformer.wte(idx)
        pos_emb = self.transformer.wpe(pos)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        x = x.mean(dim=1)  # Take the mean embedding of the tokens
        logits = self.sentiment_head(x)  # Geting logits for 3-class classification

        loss = None
        if targets is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, targets)

        return logits, loss


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
import nlpaug.augmenter.word as naw
from nltk.stem import WordNetLemmatizer
from nltk.corpus import RegexpTokenizer, stopwords
import string
nltk.download('stopwords')
# Creating dataloader for our dataset
# Dataloader contains preprocess, augment, padding all these

class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_length, padding=None, augment=False,take_only_customer=False,if_preprocess=True):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_to_int = {"negative": 0, "neutral": 1, "positive": 2}
        if padding is None:
            self.padding = "max_length"
        else:
            self.padding = False
        self.augment = augment
        self.if_take_customer = take_only_customer
        self.if_preprocess = if_preprocess
        if augment:
            self.aug = naw.SynonymAug(aug_src='wordnet')

        if self.if_preprocess:
          self.df = self.preprocess(self.df)

    def __len__(self):
        return len(self.df)

    # Remove Punctuation
    def remove_punctuation(self,text):
        no_punct = "".join([c for c in text if c not in string.punctuation])
        return no_punct

    # Remove Stopwords
    def remove_stopwords(self,text):
        words = [w for w in text if w not in stopwords.words('english')]
        return words

    def word_lemmatizer(self,text):
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(x) for x in text]

    def remove_numbers(self,text):
        return [x for x in text if not x.isdigit()]


    def take_only_customer(self,text):
        lines = text.strip().split("\n")
        customer_lines = [line.replace("Customer: ", "") for line in lines if line.startswith("Customer:")]
        customer_conversation = "\n".join(customer_lines)
        return customer_conversation

    def preprocess(self,df):
        df["conversation"] = df["conversation"].apply(lambda x: self.remove_punctuation(x))
        # Tokenize the text
        tokenizer = RegexpTokenizer(r'\w+')
        df["conversation"] = df["conversation"].apply(lambda x: tokenizer.tokenize(x.lower()))
        df["conversation"] = df["conversation"].apply(lambda x: self.remove_stopwords(x))
        df["conversation"] = df["conversation"].apply(lambda x: self.word_lemmatizer(x))
        df["conversation"] = df["conversation"].apply(lambda x: self.remove_numbers(x))

        df["conversation"] = df["conversation"].apply(lambda x: " ".join(x))
        return df

    def __getitem__(self, idx):

        row = self.df.iloc[idx]
        text = row["conversation"]
        if self.if_take_customer:
           text = self.take_only_customer(text)
        if self.augment:
           text = self.aug.augment(text)[0]
        sentiment_label = self.label_to_int[row["customer_sentiment"]]
        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_length,
          return_token_type_ids=False,
          padding=self.padding,
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
        )
        if len(encoding['input_ids'].flatten()) == 0:
          return {
          'input_ids': torch.tensor([0]),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(sentiment_label, dtype=torch.long)
          }
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(sentiment_label, dtype=torch.long)
        }


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Training And Evaluation (From Scratch And Pretrained GPT-2) 

In [7]:
# Settings
out_dir = 'out'
eval_interval = 20
eval_test_interval = 100
log_interval = 1
eval_iters = 200
if_SMOTE = True
eval_only = False
always_save_checkpoint = True
wandb_log = True
wandb_project = 'DI725-Asg1'

gradient_accumulation_steps = 5 * 8
batch_size = 1
block_size = 1024
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.0
bias = False
learning_rate = 3e-5
max_iters = 500
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0
decay_lr = False
warmup_iters = 2000
lr_decay_iters = 600000
min_lr = 6e-5
backend = 'nccl'
device = "cuda"
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
compile = False

# Collect config for wandb logging
config_keys = [k for k, v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys}


In [8]:
def configure_optimizers(model, weight_decay, learning_rate, betas, device_type):
    param_dict = {pn: p for pn, p in model.named_parameters() if p.requires_grad}
    decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
    nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
    optim_groups = [
        {'params': decay_params, 'weight_decay': weight_decay},
        {'params': nodecay_params, 'weight_decay': 0.0}
    ]
    fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
    use_fused = fused_available and device_type == 'cuda'
    extra_args = dict(fused=True) if use_fused else dict()
    optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
    return optimizer


In [9]:
def setup_model_and_data(init_from, wandb_run_name):
    ddp = int(os.environ.get('RANK', -1)) != -1
    if ddp:
        init_process_group(backend=backend)
        ddp_rank = int(os.environ['RANK'])
        ddp_local_rank = int(os.environ['LOCAL_RANK'])
        ddp_world_size = int(os.environ['WORLD_SIZE'])
        device_local = f'cuda:{ddp_local_rank}'
        torch.cuda.set_device(device_local)
        master_process = ddp_rank == 0
        seed_offset = ddp_rank
        assert gradient_accumulation_steps % ddp_world_size == 0
    else:
        device_local = device
        master_process = True
        seed_offset = 0
        ddp_world_size = 1

    torch.manual_seed(1337 + seed_offset)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    device_type_local = 'cuda' if 'cuda' in device_local else 'cpu'
    ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
    ctx = nullcontext() if device_type_local == 'cpu' else torch.amp.autocast(device_type=device_type_local, dtype=ptdtype)

    data_dir = os.path.join('data', "random")
    meta_vocab_size = None
    meta_path = os.path.join(data_dir, 'meta.pkl')
    if os.path.exists(meta_path):
        with open(meta_path, 'rb') as f:
            meta = pickle.load(f)
        meta_vocab_size = meta['vocab_size']

    model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                      bias=bias, vocab_size=None, dropout=dropout)

    if init_from == 'scratch':
        print("Initializing from scratch")
        model_args['vocab_size'] = meta_vocab_size if meta_vocab_size else 50304
        gptconf = GPTConfig(**model_args)
        model_gpt = GPT(gptconf)
        model = GPTSentiment(gptconf)
    elif init_from == 'resume':
        ckpt_path = os.path.join(out_dir, 'ckpt.pt')
        checkpoint = torch.load(ckpt_path, map_location=device_local)
        checkpoint_model_args = checkpoint['model_args']
        for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
            model_args[k] = checkpoint_model_args[k]
        gptconf = GPTConfig(**model_args)
        model_gpt = GPT(gptconf)
        model_gpt.load_state_dict(checkpoint['model'])
        model = GPTSentiment(model_gpt)
    elif init_from.startswith('gpt2'):
        print(f"Initializing from GPT2: {init_from}")
        override_args = dict(dropout=dropout)
        model_gpt = GPT.from_pretrained(init_from, override_args)
        for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
            model_args[k] = getattr(model_gpt.config, k)
        gptconf = GPTConfig(**model_args)
        model = GPTSentiment(gptconf)
        model.load_state_dict(model_gpt.state_dict(), strict=False)

    model.to(device_local)
    scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))
    optimizer = configure_optimizers(model, weight_decay, learning_rate, (beta1, beta2), device_type_local)

    if compile:
        model = torch.compile(model)

    if ddp:
        model = DDP(model, device_ids=[ddp_local_rank])

    if wandb_log and master_process:
        import wandb
        wandb.init(project=wandb_project, name=wandb_run_name, config=config)

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    train_df = pd.read_csv('./data/customer_service/train.csv')
    test_df = pd.read_csv('./data/customer_service/test.csv')
    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

    if if_SMOTE:
        sm = RandomOverSampler(random_state=42)
        X_train_smote, y_train_smote = sm.fit_resample(train_df[["conversation"]], train_df["customer_sentiment"])
        train_df_smoted = pd.concat([X_train_smote, y_train_smote.reset_index(drop=True)], axis=1)
    else:
        train_df_smoted = train_df

    train_dataset = SentimentDataset(train_df_smoted, tokenizer, max_length=512, padding=False, augment=True, take_only_customer=False, if_preprocess=True)
    val_dataset = SentimentDataset(val_df, tokenizer, max_length=512, padding=False, take_only_customer=False, if_preprocess=True)
    test_dataset = SentimentDataset(test_df, tokenizer, max_length=512, padding=False, take_only_customer=False, if_preprocess=True)

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    return model, model_args, optimizer, scaler, ctx, train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset, device_local


In [10]:
def estimate_loss(model, ctx, train_loader_iter, val_loader_iter,
                  train_loader, val_loader, num_train, num_val, device):
    out = {"train": {}, "val": {}}
    model.eval()
    all_preds = []
    all_labels = []

    for split in ['train', 'val']:
        loader_iter = train_loader_iter if split == "train" else val_loader_iter
        loader = train_loader if split == "train" else val_loader
        dataset_len = num_train if split == "train" else num_val

        iter_num = dataset_len if loader.batch_size == 1 else min(dataset_len // loader.batch_size + 1, eval_iters // loader.batch_size + 1)
        losses = torch.zeros(iter_num)

        for k in range(iter_num):
            try:
                batch = next(loader_iter)
                X = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                Y = batch['labels'].to(device)
            except:
                continue

            with ctx:
                logits, loss = model(X, Y)
                preds = torch.argmax(logits, dim=-1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(Y.cpu().numpy())
            losses[k] = loss.item()

        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        conf_matrix = confusion_matrix(all_labels, all_preds)

        out[split] = {
            "loss": losses.mean(),
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "conf_matrix": conf_matrix
        }

    model.train()
    return out

def estimate_loss_test(model, ctx, test_loader, num_samples, device):
    out = {"test": {}}
    test_loader_iter = iter(test_loader)
    model.eval()
    all_preds = []
    all_labels = []

    iter_num = num_samples if test_loader.batch_size == 1 else min(num_samples // test_loader.batch_size + 1, eval_iters // test_loader.batch_size + 1)
    losses = torch.zeros(iter_num)

    for k in range(iter_num):
        try:
            batch = next(test_loader_iter)
            X = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            Y = batch['labels'].to(device)
        except:
            continue

        with ctx:
            logits, loss = model(X, Y)
            preds = torch.argmax(logits, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(Y.cpu().numpy())
        losses[k] = loss.item()

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    out["test"] = {
        "loss": losses.mean(),
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "conf_matrix": conf_matrix
    }

    model.train()
    return out, all_labels, all_preds



In [11]:
def train_model(init_from):
    global config
    wandb_run_name = 'gpt2_' + init_from
    model, model_args, optimizer, scaler, ctx, train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset, device_used = setup_model_and_data(init_from, wandb_run_name)

    train_loader_iter = iter(train_loader)
    val_loader_iter = iter(val_loader)
    test_loader_iter = iter(test_loader)

    t0 = time.time()
    local_iter_num = 0
    iter_num = 0
    best_val_loss = 1e9
    train_loss_metric = MeanMetric()
    val_loss_metric = MeanMetric()
    raw_model = model.module if isinstance(model, DDP) else model

    batch = next(train_loader_iter)
    X = batch['input_ids'].to(device_used)
    attention_mask = batch['attention_mask'].to(device_used)
    Y = batch['labels'].to(device_used)

    while True:
        lr = get_lr(iter_num) if decay_lr else learning_rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        if iter_num % eval_interval == 0:
            out_res = estimate_loss(
                model, ctx,
                train_loader_iter, val_loader_iter,
                train_loader, val_loader,
                len(train_dataset), len(val_dataset),
                device_used
            )
            print(f"step {iter_num}: train loss {train_loss_metric.compute():.4f}, val loss {val_loss_metric.compute():.4f}")
            print(f"step val accuracy {out_res['val']['accuracy']:.4f}")

            if wandb_log:
                wandb.log({
                    "iter": iter_num,
                    "train/loss": train_loss_metric.compute(),
                    "val/loss": val_loss_metric.compute(),
                    "lr": lr,
                    "val/acc": out_res['val']['accuracy'],
                })

            if out_res['val']["loss"] < best_val_loss or always_save_checkpoint:
                best_val_loss = out_res['val']["loss"]
                if iter_num > 0:
                    checkpoint = {
                        'model': raw_model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'model_args': model_args,
                        'iter_num': iter_num,
                        'best_val_loss': best_val_loss,
                        'config': config,
                    }
                    os.makedirs(out_dir, exist_ok=True)
                    print(f"saving checkpoint to {out_dir}")
                    torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))

        if iter_num % eval_test_interval == 0:
            out_res, all_labels, all_preds = estimate_loss_test(model, ctx, test_loader, len(test_dataset), device_used)
            test_acc = out_res['test']['accuracy']
            test_precision = out_res['test']['precision']
            test_recall = out_res['test']['recall']
            test_f1 = out_res['test']['f1']
            print(f"Test accuracy {test_acc:.4f}")
            print(f"Test precision {test_precision:.4f}")
            print(f"Test recall {test_recall:.4f}")
            print(f"Test F1 {test_f1:.4f}")
            print("Confusion Matrix")
            print("------------------")
            print(out_res['test']['conf_matrix'])

            if wandb_log:
                wandb.log({
                    "Test_Accuracy": test_acc,
                    "Test_Precision": test_precision,
                    "Test_Recall": test_recall,
                    "Test_F1_Score": test_f1
                })
                x_labels = ["negative", "neutral", "positive"]
                wandb.log({
                    "Confusion Matrix": wandb.plot.confusion_matrix(
                        probs=None,
                        y_true=all_labels,
                        preds=all_preds,
                        class_names=x_labels
                    )
                })

        if iter_num == 0 and eval_only:
            break

        for micro_step in range(gradient_accumulation_steps):
            if isinstance(model, DDP):
                model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
            with ctx:
                logits, loss = model(X, Y)
                train_loss_metric.update(loss.item())
                loss = loss / gradient_accumulation_steps
            try:
                batch = next(train_loader_iter)
            except StopIteration:
                train_loader_iter = iter(train_loader)
                batch = next(train_loader_iter)

            X = batch['input_ids'].to(device_used)
            attention_mask = batch['attention_mask'].to(device_used)
            Y = batch['labels'].to(device_used)

            scaler.scale(loss).backward()

        if grad_clip != 0.0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

        model.eval()
        with torch.no_grad():
            try:
                batch_val = next(val_loader_iter)
            except StopIteration:
                val_loader_iter = iter(val_loader)
                batch_val = next(val_loader_iter)
            X_val = batch_val['input_ids'].to(device_used)
            Y_val = batch_val['labels'].to(device_used)
            logits_val, loss_val = model(X_val, Y_val)
            val_loss_metric.update(loss_val.item())
        model.train()

        t1 = time.time()
        dt = t1 - t0
        t0 = t1
        if iter_num % log_interval == 0:
            lossf = loss.item() * gradient_accumulation_steps
            print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")

        iter_num += 1
        local_iter_num += 1

        if iter_num > max_iters:
            break


In [ ]:
# First run: From Scratch
train_model("scratch")
# Second run: From GPT-2 pretrained
train_model("gpt2")

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


tokens per iteration will be: 40,960
Initializing a new model from scratch
defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)
number of parameters: 123.59M
number of parameters: 123.59M
num decayed parameter tensors: 51, with 124,356,864 parameters
num non-decayed parameter tensors: 26, with 19,203 parameters
using fused AdamW: True


<ipython-input-6-763ebe6df08c>:243: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aliyigitbasaran (aliyigitbasaran-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MeanMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


step 0: train loss nan, val loss nan
step val accuracy 0.2943
Number of Samples Test =  30


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test accuracy 0.3333
Test precision 0.1149
Test recall 0.3333
Test F1 0.1709
Confusion Matrix
------------------
[[ 0  0 10]
 [ 1  0  9]
 [ 0  0 10]]
iter 0: loss 1.3125, time 69357.87ms
iter 1: loss 11.3125, time 2794.59ms
iter 2: loss 0.2080, time 2814.23ms
iter 3: loss 2.8594, time 3199.42ms
iter 4: loss 0.7461, time 3235.71ms
iter 5: loss 0.7891, time 3001.80ms
iter 6: loss 1.7734, time 2991.91ms
iter 7: loss 1.3359, time 2973.93ms
iter 8: loss 1.5078, time 3199.47ms
iter 9: loss 1.1563, time 3029.41ms
iter 10: loss 1.7031, time 2839.33ms
iter 11: loss 1.7656, time 2972.85ms
iter 12: loss 1.3281, time 3122.76ms
iter 13: loss 1.3984, time 2880.09ms
iter 14: loss 0.4785, time 2829.61ms
iter 15: loss 1.8594, time 2891.76ms
iter 16: loss 1.4766, time 3030.59ms
iter 17: loss 0.7773, time 3346.64ms
iter 18: loss 1.9766, time 3533.66ms
iter 19: loss 1.5156, time 3111.10ms
step 20: train loss 1.5760, val loss 1.2840
step val accuracy 0.5089
saving checkpoint to out
iter 20: loss 1.3281, ti

# Hyperparameter Tuning for Pretrianed GPT-2 

In [12]:
# Rebuild datasets with hyper config for augment and preprocessing.
# (We assume that SentimentDataset accepts 'augment' and 'if_preprocess' as parameters.)
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

def setup_model_and_data_hyper(init_from, wandb_run_name, if_SMOTE_override=None, augment_override=None, if_preprocess_override=None):
    """
    This is a wrapper around your existing setup_model_and_data
    that overrides some data parameters based on wandb hyperconfig.
    """
    # Override global data parameters if provided
    global if_SMOTE
    if if_SMOTE_override is not None:
        if_SMOTE = if_SMOTE_override  # use the hyper parameter value
    
    # Call the original setup (which is assumed to use globals for SMOTE, augment, and preprocessing)
    model, model_args, optimizer, scaler, ctx, train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset, device_used = setup_model_and_data(init_from, wandb_run_name)

    train_df = pd.read_csv('./data/customer_service/train.csv')
    test_df = pd.read_csv('./data/customer_service/test.csv')
    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
    
    if if_SMOTE:
        from imblearn.over_sampling import RandomOverSampler
        sm = RandomOverSampler(random_state=42)
        X_train_smote, y_train_smote = sm.fit_resample(train_df[["conversation"]], train_df["customer_sentiment"])
        train_df_smoted = pd.concat([X_train_smote, y_train_smote.reset_index(drop=True)], axis=1)
    else:
        train_df_smoted = train_df

    # Use wandb config overrides for augmentation and preprocessing; default to True if not provided.
    train_dataset = SentimentDataset(
        train_df_smoted, tokenizer, max_length=512, padding=False,
        augment=augment_override if augment_override is not None else True,
        take_only_customer=False,
        if_preprocess=if_preprocess_override if if_preprocess_override is not None else True
    )
    val_dataset = SentimentDataset(
        val_df, tokenizer, max_length=512, padding=False,
        take_only_customer=False,
        if_preprocess=if_preprocess_override if if_preprocess_override is not None else True
    )
    test_dataset = SentimentDataset(
        test_df, tokenizer, max_length=512, padding=False,
        take_only_customer=False,
        if_preprocess=if_preprocess_override if if_preprocess_override is not None else True
    )
    from torch.utils.data import DataLoader
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    return model, model_args, optimizer, scaler, ctx, train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset, device_used


In [13]:
# === Modularized Hyperparameter Tuning Function ===
def hyper_train_model():
    """
    This function is designed for hyperparameter tuning runs using GPT-2 weights.
    It is invoked via wandb sweep and reuses your earlier setup and evaluation functions.
    """
    import wandb, time, math, os
    from datetime import datetime
    with wandb.init() as run:
        wandb_config = wandb.config
        # Use wandb sweep parameters for data augmentation, SMOTE, and preprocessing
        if_smote = wandb_config.get("if_smote", True)
        augment_flag = wandb_config.get("augment", True)
        if_preprocess_flag = wandb_config.get("if_preprocess", True)
        
        # Set up parameters for this hyper run (only using GPT2 weights)
        init_from = "gpt2"  
        now = datetime.now()
        formatted_time = now.strftime("%H%M")
        print(formatted_time)
        wandb_run_name = wandb_config.get("run_name", f"gpt2_hyper_{formatted_time}")
        
        # Call modified setup to get the model, optimizer, data loaders, etc.
        model, model_args, optimizer, scaler, ctx, train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset, device_used = setup_model_and_data_hyper(
            init_from, wandb_run_name, 
            if_SMOTE_override=if_smote, 
            augment_override=augment_flag, 
            if_preprocess_override=if_preprocess_flag
        )
        
        # Define a learning rate scheduler
        def get_lr(it):
            warmup_iters = wandb_config.get("warmup_iters", 2000)
            lr_decay_iters = wandb_config.get("lr_decay_iters", 600000)
            base_lr = wandb_config["learning_rate"]
            min_lr = wandb_config.get("min_lr", 6e-5)
            if it < warmup_iters:
                return base_lr * it / warmup_iters
            if it > lr_decay_iters:
                return min_lr
            decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
            coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
            return min_lr + coeff * (base_lr - min_lr)
        
        # Initialize iterators and metrics
        from torchmetrics import MeanMetric
        train_loader_iter = iter(train_loader)
        val_loader_iter = iter(val_loader)
        t0 = time.time()
        iter_num = 0
        best_val_loss = float("inf")
        train_loss_metric = MeanMetric()
        val_loss_metric = MeanMetric()
        raw_model = model.module if isinstance(model, torch.nn.parallel.DistributedDataParallel) else model

        # Grab an initial training batch
        batch = next(train_loader_iter)
        X = batch['input_ids'].to(device_used)
        Y = batch['labels'].to(device_used)

        max_iters = wandb_config.get("max_iters", 500)
        eval_interval = wandb_config.get("eval_interval", 20)
        eval_test_interval = wandb_config.get("eval_test_interval", 100)
        gradient_accumulation_steps = wandb_config.get("gradient_accumulation_steps", 40)
        grad_clip = wandb_config.get("grad_clip", 1.0)
        log_interval = wandb_config.get("log_interval", 1)

        # Main training loop
        while iter_num <= max_iters:
            lr = get_lr(iter_num) if wandb_config.get("decay_lr", False) else wandb_config["learning_rate"]
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

            # Evaluation on train and validation sets at intervals
            if iter_num % eval_interval == 0:
                # Use your previously defined estimate_loss function
                out_res = estimate_loss(model, ctx, train_loader_iter, val_loader_iter, train_loader, val_loader, len(train_dataset), len(val_dataset), device_used)
                print(f"Step {iter_num}: train loss {train_loss_metric.compute():.4f}, val loss {val_loss_metric.compute():.4f}")
                print(f"Validation accuracy: {out_res['val']['accuracy']:.4f}")
                wandb.log({
                    "iter": iter_num,
                    "train/loss": train_loss_metric.compute(),
                    "val/loss": val_loss_metric.compute(),
                    "lr": lr,
                    "val/acc": out_res['val']['accuracy']
                })
                # Save checkpoint if validation loss improves
                if out_res['val']['loss'] < best_val_loss:
                    best_val_loss = out_res['val']['loss']
                    if iter_num > 0:
                        checkpoint = {
                            'model': raw_model.state_dict(),
                            'optimizer': optimizer.state_dict(),
                            'model_args': model_args,
                            'iter_num': iter_num,
                            'best_val_loss': best_val_loss,
                            'config': wandb_config,
                        }
                        os.makedirs("out", exist_ok=True)
                        print(f"Saving checkpoint to out/ckpt.pt")
                        torch.save(checkpoint, os.path.join("out", "ckpt.pt"))
            
            # Evaluation on the test set at intervals
            if iter_num % eval_test_interval == 0:
                out_res, all_labels, all_preds = estimate_loss_test(model, ctx, test_loader, len(test_dataset), device_used)
                print(f"Test accuracy {out_res['test']['accuracy']:.4f}")
                wandb.log({
                    "Test_Accuracy": out_res['test']['accuracy'],
                    "Test_Precision": out_res['test']['precision'],
                    "Test_Recall": out_res['test']['recall'],
                    "Test_F1_Score": out_res['test']['f1'],
                    "Confusion Matrix": wandb.plot.confusion_matrix(
                        probs=None,
                        y_true=all_labels,
                        preds=all_preds,
                        class_names=["negative", "neutral", "positive"]
                    )
                })
            
            # Forward/backward pass with gradient accumulation
            for micro_step in range(gradient_accumulation_steps):
                if isinstance(model, torch.nn.parallel.DistributedDataParallel):
                    model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
                with ctx:
                    logits, loss = model(X, Y)
                    train_loss_metric.update(loss.item())
                    loss = loss / gradient_accumulation_steps
                try:
                    batch = next(train_loader_iter)
                except StopIteration:
                    train_loader_iter = iter(train_loader)
                    batch = next(train_loader_iter)
                X = batch['input_ids'].to(device_used)
                Y = batch['labels'].to(device_used)
                scaler.scale(loss).backward()
            
            # Gradient clipping if enabled
            if grad_clip != 0.0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            
            # Do a quick validation forward pass for metric update
            model.eval()
            with torch.no_grad():
                try:
                    batch_val = next(val_loader_iter)
                except StopIteration:
                    val_loader_iter = iter(val_loader)
                    batch_val = next(val_loader_iter)
                X_val = batch_val['input_ids'].to(device_used)
                Y_val = batch_val['labels'].to(device_used)
                logits_val, loss_val = model(X_val, Y_val)
                val_loss_metric.update(loss_val.item())
            model.train()
            
            # Logging timing and loss info
            t1 = time.time()
            dt = t1 - t0
            t0 = t1
            if iter_num % log_interval == 0:
                lossf = loss.item() * gradient_accumulation_steps
                print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
            iter_num += 1
        wandb.finish()



In [14]:
# === Sweep configuration and agent setup ===
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'Test_Accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'if_smote': {
            'values': [True, False]
        },
        'if_preprocess': {
            'values': [True, False]
        },
        'learning_rate': {
            'values': [5e-5, 1e-4]
        },
        'augment': {
            'values': [True, False]
        }
    }
}

In [15]:
wandb_project = 'DI725-Asg1-HyperParamTune'

# Create a sweep and run the agent.
sweep_id = wandb.sweep(sweep_config, project=wandb_project)
wandb.agent(sweep_id, hyper_train_model)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: zm57at65
Sweep URL: https://wandb.ai/aliyigitbasaran-/DI725-Asg1-HyperParamTune/sweeps/zm57at65


wandb: Agent Starting Run: 1qn18iv0 with config:
wandb: 	augment: True
wandb: 	if_preprocess: True
wandb: 	if_smote: True
wandb: 	learning_rate: 5e-05
wandb: Currently logged in as: aliyigitbasaran (aliyigitbasaran-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


1803
Initializing from GPT2: gpt2
loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.0
number of parameters: 123.65M
number of parameters: 123.65M


C:\Users\Ali\AppData\Local\Temp\ipykernel_24856\3085892497.py:64: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))


...

Step 60: train loss 0.5247, val loss 0.6044
Validation accuracy: 0.8823
iter 60: loss 0.0020, time 18566.45ms
iter 61: loss 0.1875, time 1592.70ms
iter 62: loss 0.0039, time 1645.41ms
iter 63: loss 0.0248, time 1583.07ms
iter 64: loss 0.0008, time 1660.67ms
iter 65: loss 0.0171, time 1660.39ms
iter 66: loss 0.0415, time 1572.77ms
iter 67: loss 0.0016, time 1625.58ms
iter 68: loss 0.0017, time 1626.10ms
iter 69: loss 0.0038, time 1628.72ms
iter 70: loss 0.0008, time 1572.07ms
iter 71: loss 0.0613, time 1596.71ms
iter 72: loss 2.5781, time 1670.93ms
iter 73: loss 0.0012, time 1593.59ms
iter 74: loss 0.0167, time 1578.76ms
iter 75: loss 0.0405, time 1612.30ms
iter 76: loss 0.0075, time 1646.41ms
iter 77: loss 0.4434, time 1637.97ms
iter 78: loss 0.0128, time 1811.39ms
iter 79: loss 0.0220, time 1665.39ms


c:\Users\Ali\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ali\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Step 80: train loss 0.4291, val loss 0.5531
Validation accuracy: 0.9546
iter 80: loss 0.0020, time 18616.73ms
iter 81: loss 1.8828, time 1643.11ms
iter 82: loss 0.0016, time 1588.81ms
iter 83: loss 0.0036, time 1624.30ms
iter 84: loss 0.0015, time 1655.09ms
iter 85: loss 0.0006, time 1596.83ms
iter 86: loss 0.0004, time 1607.97ms
iter 87: loss 0.4883, time 1638.65ms
iter 88: loss 0.0408, time 1566.74ms
iter 89: loss 0.0011, time 1656.29ms
iter 90: loss 0.0172, time 1675.66ms
iter 91: loss 0.0005, time 1612.11ms
iter 92: loss 0.0010, time 1613.51ms
iter 93: loss 0.0006, time 1604.10ms
iter 94: loss 0.0038, time 1573.12ms
iter 95: loss 0.0001, time 1624.71ms
iter 96: loss 0.0010, time 1570.24ms
iter 97: loss 0.0004, time 1677.07ms
iter 98: loss 0.0003, time 1579.10ms
iter 99: loss 0.0001, time 1633.78ms
Step 100: train loss 0.3555, val loss 0.5557
Validation accuracy: 0.9438
Test accuracy 0.6667
iter 100: loss 0.1416, time 19684.34ms
iter 101: loss 0.0001, time 1608.48ms
iter 102: loss 0

Test_Accuracy,▁▅▇▆▆█
Test_F1_Score,▁▆▇▇▆█
Test_Precision,▁▆█▇▇█
Test_Recall,▁▅▇▆▆█
iter,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val/acc,▁▆▇▇█▇█████▇██████████████
val/loss,▇▅▃▁▁▂▁▁▂▃▃▄▅▄▅▅▅▆▇█████▇
Test_Accuracy,0.9
Test_F1_Score,0.89975


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
